# Создание файла в формате parquet

В этом примере используются библиотеки spark.
Библиотеки pandas не возможно использовать на объемах более одного гигабайта данных.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, LongType, StringType, IntegerType, DateType, TimestampType, FloatType, BooleanType
from pyspark.sql.functions import col, cast, date_trunc, sum, dayofweek, hour, dayofmonth, lit, sequence

import random

In [ ]:
# запуск spark
spark = SparkSession.builder.appName('parquet').getOrCreate()

In [ ]:

schemaData = StructType([ \
    StructField("label",StringType(),True),
    StructField("feature1",IntegerType(),True), 
    StructField("feature2",FloatType(),True), 
    StructField("feature3",BooleanType(),True), 
  ])

pathParquet = "/tmp/parquetFiles"

In [ ]:
# формирование dataframe
values= []
df = spark.createDataFrame(values,schema=schemaData)
df.printSchema()
df.show()
# запишем пустые данные, необходимо для очистки
df.write.mode("overwrite").parquet(pathParquet)

In [ ]:
# убедимся, что папка создана, данных нет
dfCount = spark.read.parquet(pathParquet)
dfCount.count()

In [ ]:
schemaIndex = StructType([ \
    StructField("indx1",LongType(),True),
    StructField("indx2",LongType(),True),
  ])

In [ ]:
# создаем пачку данных
# кажется, что гораздо проще создать готовый список и затем обернуть его в spark,
# но в отличии от spark простой список не поместится в памяти
N = 10
indx = list((idx,idx) for idx in range(N))
df = spark.createDataFrame(indx, schemaIndex)

def extract_features(row):
 return (
    'label'+str(random.randint(1, 10)), #label
    random.randint(1, N), #feature1
    random.random(), #feature2
    bool(random.getrandbits(1)) , #feature3
 )

rdd = df.rdd.map(extract_features)
df = rdd.toDF( ["label","feature1","feature2", "feature3"])
df.show()
# запись в parquet
df.write.mode("append").parquet(pathParquet)

In [ ]:
# проверяем количество
dfCount = spark.read.parquet(pathParquet)
dfCount.count()

In [ ]:
# увеличиваем набор данных
batchSize = 30
labelSize = 10
def writeParquet():
    indx = list((idx,idx) for idx in range(batchSize))
    df = spark.createDataFrame(indx, schemaIndex)
    def extract_features(row):
        return (
            'label'+str(random.randint(1, labelSize)), #label
            random.randint(1, N), #feature1
            random.random(), #feature2
            bool(random.getrandbits(1)) , #feature3
        )
    rdd = df.rdd.map(extract_features)
    df = rdd.toDF( ["label","feature1","feature2", "feature3"])
    # запись в parquet
    df.write.mode("append").parquet(pathParquet)


# делаем некоторое количество пачек
for i in range(5):
    writeParquet()

# проверяем количество
dfCount = spark.read.parquet(pathParquet)
dfCount.count()

В папке будет создано множество мелких файлов, количество файлов будет зависеть от числа используемых процессоров, по одному файлу на процессор, плюс несколько итераций. Можно создать один файл. При объединении нужно учесть, один файл может создаваться только одной ниткой процессора, а следовательно время ожидания будет на много выше.

In [ ]:
localPathParquet = "/tmp/singleparquet"
dfSingle = spark.read.parquet(pathParquet)
fileCount = 1 # число файлов
dfSingle.coalesce(1).write.mode("overwrite").parquet(localPathParquet)

In [ ]:
spark.stop()

In [ ]:
# полученный набор данных (dataset) можно закинуть на S3
import os
import boto3

# имя файла в s3
filePathParquet = "/tmp/dataset.parquet"
# имя корзины в s3
bucketNameParquet = 'data'

s3_target = boto3.resource('s3', 
    endpoint_url=os.environ["AWS_ENDPOINT_URL"],
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)

# создание корзины
bucket = s3_target.Bucket(bucketNameParquet)
if bucket.creation_date:
   print("The bucket exists")
else:
   print("The bucket does not exist")
   s3_target.create_bucket(Bucket=bucketNameParquet)

# загрузка файла на s3 
# в локальной папке находится единственный файл и закидывается на S3
print("старт записи в s3")
fileNameParquetLocal = [x for x in os.listdir(localPathParquet) if x.endswith(".parquet")][0]
print('write to s3', 'backet='+bucketNameParquet, 'path='+filePathParquet)
s3_target.Bucket(bucketNameParquet).upload_file(localPathParquet+'/'+fileNameParquetLocal, filePathParquet)